In [ ]:
import importlib

In [ ]:
import MecaClass
importlib.reload(MecaClass)
from MecaClass import Meca

In [ ]:
m = Meca()
m.connect()
m.home()

In [ ]:
points = (170, 10, 80, 180, 0, 0)
m.move_lin(points)

In [ ]:
m.disconnect()